# Problem statment

given an array 'arr' of elements find the subset with maximum sum, less than 'max_sum'

In [1]:
import jax
import jax.numpy as jnp
from jax import grad, value_and_grad
import random
import time

In [37]:
def softmax(x):
    e_x = jnp.exp(x - jnp.max(x))
    return e_x / e_x.sum()

def get_sum(arr, str):
    # softmax the strength to 0-1
    strength = jnp.array(arr)
    strength = softmax(strength)
    strength = [1 if s > 0.5 else 0 for s in strength]
    return jnp.sum(jnp.array(strength) * jnp.array(str))

def loss_fn(arr, str, max_sum):
    return jnp.abs(get_sum(arr, str) - max_sum)

In [40]:
def lr_schduler(i):
    return 0.1 

def solve(arr, max_sum):
    arr = jnp.array(arr)
    iterations_num = 1000
    strength = jnp.zeros(len(arr))
    loss_value_and_grad = value_and_grad(loss_fn, 1)

    for i in range(iterations_num):
        loss, grad = loss_value_and_grad(arr, strength, max_sum)
        strength -= lr_schduler(i) * grad
        if i % 100 == 0:
            print(f'iteration {i}, loss {loss}')

    print(f'final loss {loss}')
    strength = softmax(strength)
    strength = [1 if s > 0.5 else 0 for s in strength]
    print(f'final strength {strength}')
    print(f'final sum {get_sum(arr, strength)}')
   


In [41]:
random.seed(420)
max_sum = 14
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
arr = random.sample(arr, len(arr))
print(arr)

solve(arr, max_sum)

[1, 6, 5, 4, 10, 9, 8, 3, 2, 7]
iteration 0, loss 14.0
iteration 100, loss 3.999998092651367
iteration 200, loss 1.71661376953125e-05
iteration 300, loss 1.71661376953125e-05
iteration 400, loss 1.71661376953125e-05
iteration 500, loss 1.71661376953125e-05
iteration 600, loss 1.71661376953125e-05
iteration 700, loss 1.71661376953125e-05
iteration 800, loss 1.71661376953125e-05
iteration 900, loss 1.71661376953125e-05
final loss 0.09998321533203125
final strength [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
final sum 1
